In [11]:
import numpy as np
import os
from PIL import Image
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import cv2

In [12]:
filepath ="/home/user/yang/"
#filepath ="/home/user/Downloads/211116_Yang_source_code/"

In [13]:
#get atom 2D coordination from 2d_xyz_file
os.chdir(filepath + "sorce_code/example_data/xyz_file/2d_xyz")###change to the file directory where you save 2D xyz files

xyz_file_list=os.listdir(".")
xyz_file_list.sort(key=lambda x:x[:-2])
len(xyz_file_list)
#同样测试用
#xyz_file_list = xyz_file_list[0:5]

871

In [14]:
#get GSH reaction probability and fukui value from result file
os.chdir(filepath + "sorce_code/example_data/total")###change to the file directory where you save total files,which generate from read_all_information

result_file_list=os.listdir(".")
result_file_list.sort(key=lambda x:x[:-2])
len(result_file_list)
#测试图像用
#result_file_list = result_file_list[0:5]
len(result_file_list)

871

In [15]:
def calculat_gaussian(x,y,m,n,k):
    sigma=0.6

    gaussian = (1/(2*np.pi*sigma**2))* \
    np.exp(-((pow(x-m, 2) + pow(y-n, 2)) \
    /(2*pow(sigma, 2))))
    gaussian=gaussian*k
    
    return gaussian
    

In [16]:
xyz_file_path = [
    os.path.join(filepath + "sorce_code/example_data/xyz_file/2d_xyz",x)###change to the file directory where you save 2D xyz files
    for x in xyz_file_list
]
result_file_path = [
    os.path.join(filepath + "sorce_code/example_data/total",x)###change to the file directory where you save total files,which generate from read_all_information
    for x in result_file_list
]

In [17]:
xyz_file_path.sort(key=lambda x:x[:-4])

In [18]:
result_file_path.sort(key=lambda x:x[:-4])

In [19]:
for i in range(len(xyz_file_list)):
    result_file =  result_file_path[i]
    xyz_file = xyz_file_path[i]
    
    coor_x = []
    coor_y = []
    with open (xyz_file,"r") as f:
        cp = f.readlines()[2:]
    for line in cp:
        line=line.split()
        coor_x.append(line[1])
        coor_y.append(line[2])
    coor_x_num=[np.float(n) for n in coor_x]
    coor_y_num=[np.float(n) for n in coor_y]
    
    x_start = min(coor_x_num)-2
    x_end = max(coor_x_num)+2
    y_start = min(coor_y_num)-2
    y_end = max(coor_y_num)+2
    start_min = min(x_start,y_start)
    end_max = max(x_end,y_end)
    x=np.linspace(start_min,end_max,400)
    y=np.linspace(start_min,end_max,400)
    X,Y=np.meshgrid(x,y)
    os.chdir(filepath + "sorce_code/example_data/total")###change to the file directory where you save total files,which generate from read_all_information
    result_file_list=os.listdir(".")
    result_file_list.sort(key=lambda x:x[:-2])
    result_filename = result_file_list[i]
    total_value = np.loadtxt(result_filename,delimiter=',')
    #GSH = total_value[:,3]*10
    GSH = total_value[:,3]*50
    #GSH= 0.5*(total_value[:,3]+total_value[:,4])*10 #平均Fukui函数#从0开始数第二列是GSH发生概率
    
    volume = []
    for j in range(len(coor_x_num)):
        x = np.float(coor_x[j])#                                  原子xy坐标输入
        y = np.float(coor_y[j])
        k = np.float(GSH[j]) #                                 输入该原子对应的记述子的值
        result = calculat_gaussian(X,Y,x,y,k)#                        把该值进行高斯扩散，得到一张图
        
        volume.append(result)
    volume = np.array(volume)
    result=np.sum(volume,axis=0)#                                       把所有的值都叠加起来
    
    

    
    result = result.clip(-2,2)
    result = (result + 2) / (4) #to normalize
    result = (result * 200)
    result = np.expand_dims(result, 2)
    
    sv = np.zeros([400, 400, 2])
    sv[:, :, 0] = 200 #you can modify value of S for HSV
    sv[:, :, 1] = 200 #you can modify value of V for HSV
    

    hsv_image = np.concatenate([result, sv], 2).astype(np.uint8)
    rgb_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)
    #image = Image.fromarray(result, 'L').convert('RGB')
    #image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = Image.fromarray(rgb_image)
    #image = image.convert('L')
    #image.show()
    #print(image.size)

    #print(re)
    #im = Image.fromarray(result)
    #im.show()
    #im = im.convert('P')
    image.save(filepath + "sorce_code/example_data/experiment_data/red_blue_fukui_value/total_target_%s.png"%(result_filename))
    image.close()
    

In [20]:
for i in range(len(xyz_file_list)):#分子的各个原子坐标，储存在xyz文件中，
    result_file =  result_file_path[i]
    xyz_file = xyz_file_path[i]
    
    coor_x = []
    coor_y = []
    with open (xyz_file,"r") as f:
        cp = f.readlines()[2:]
    for line in cp:
        line=line.split()
        coor_x.append(line[1])
        coor_y.append(line[2])
    coor_x_num=[np.float(n) for n in coor_x]
    coor_y_num=[np.float(n) for n in coor_y]
    
    x_start = min(coor_x_num)-2
    x_end = max(coor_x_num)+2
    y_start = min(coor_y_num)-2
    y_end = max(coor_y_num)+2
    start_min = min(x_start,y_start)
    end_max = max(x_end,y_end)
    x=np.linspace(start_min,end_max,400)
    y=np.linspace(start_min,end_max,400)
    X,Y=np.meshgrid(x,y)
    os.chdir(filepath + "sorce_code/example_data/total")
    result_file_list=os.listdir(".")
    result_file_list.sort(key=lambda x:x[:-2])
    result_filename = result_file_list[i]
    total_value = np.loadtxt(result_filename,delimiter=',')
    #GSH = total_value[:,2]*10#从0开始数第二列是GSH发生概率
    GSH = total_value[:,2]*50
    #GSH= 0.5*(total_value[:,3]+total_value[:,4])*10 #平均赴京函数
    
    volume = []
    for j in range(len(coor_x_num)):
        x = np.float(coor_x[j])#原子xy坐标输入
        y = np.float(coor_y[j])
        k = np.float(GSH[j]) #输入该原子对应的记述子的值
        result = calculat_gaussian(X,Y,x,y,k)#把该值进行高斯扩散，得到一张图
        
        volume.append(result)
    volume = np.array(volume)
    result=np.sum(volume,axis=0)#把所有的值都叠加起来
    
    


    mask_background_condition_one = result < 0.1#标记其中小于0.005的值
    mask_background_condition_two = result > 0
    mask_background = mask_background_condition_one * mask_background_condition_two
    result[mask_background] = 0#让这些值全都变成0
    


    #result_filename = result_filename[21:]
    mask_4 = result > 0
    mask_5 = result <= 0.6
    mask_6 = mask_4*mask_5
    result[mask_6] = 0.15
    

    mask_19 = result > 0.6

    result[mask_19] = 0.65
    key = np.unique(result)
    result *= 255
    im = Image.fromarray(result)
    im = im.convert('L')
    im.save(filepath + "sorce_code/example_data/experiment_data/black_GSH_value/total_target_%s.png"%(result_filename))
    im.close()